<a href="https://colab.research.google.com/github/salarbalou/ThorLabs_UV_FL_Data/blob/main/ThorLabs_UV_FL_Analyisis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import requests as req
import regex as re
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
from scipy import signal
pio.templates.default = 'plotly_white'

In [3]:
baseurl = 'https://raw.githubusercontent.com/salarbalou/ThorLabs_UV_FL_Data/main/'
char = '.csv'

In [4]:
#generating a data for standard solution (water)
solution = 'water'
concentration = 'pure'
excitation = 'white'
source_water = req.get(baseurl+solution+'_'+concentration+'_'+excitation+char).text
datalist = (re.compile(r'\n([0-9.e+-]+)[,]([0-9.e+-]+)', re.MULTILINE)).findall(source_water)
datas = []
for tuples in datalist:
  eachtuple = []
  for ab in tuples:
    eachtuple.append(float(ab))
  datas.append((eachtuple[0], eachtuple[1]))
data = np.array(datas)
wavelength = data[:, 0]
intensity = data[:, 1]

In [5]:
#generating a data for experimental solution 
# since excitation must be equal for water and solution, thus previous value of excitation will be used
# Absorbance
solution = input() # like water, CQD, AuNP
concentration = input() #pure for water, value for other solutions
excitation = 'white' # wavelength of the excitation like 370 , 450
source_sol = req.get(baseurl+solution+'_'+concentration+'_'+excitation+char).text
datalist_sol = (re.compile(r'\n([0-9.e+-]+)[,]([0-9.e+-]+)', re.MULTILINE)).findall(source_sol)
datas_sol= []
for tuples_sol in datalist_sol:
  eachtuple_sol = []
  for ab in tuples_sol:
    eachtuple_sol.append(float(ab))
  datas_sol.append((eachtuple_sol[0], eachtuple_sol[1]))
data_sol = np.array(datas_sol)
wavelength_sol = data_sol[:, 0]
intensity_sol = data_sol[:, 1]
absorbance_list = data[:, 1]- data_sol[:, 1]
absorbance_list[absorbance_list<0] = 0
figure = make_subplots()
figure.add_trace(go.Scatter(x = wavelength_sol, y = signal.savgol_filter(absorbance_list, 53, 3)))
figure.update_xaxes(title = 'Absorbance Wavelength (nm)', ticks = 'outside', showgrid = False, showline = True, linecolor = 'black', range = [200, 1000], nticks = 50)
figure.update_yaxes(title = 'Absorbance Intensity',  ticks = 'outside', showgrid = False, showline = True, linecolor = 'black')
figure.update_layout(title = f'Absorbance of {solution} at {concentration} concentration excited in range 200-1000nm')

greendye
pure


In [6]:
#Fluorescent measurment 
solution = input() # like water, CQD, AuNP
concentration = input() #pure for water, value for other solutions
absorbance = input()
source_sol_FL = req.get(baseurl+solution+'_'+concentration+'_'+absorbance+char).text
datalist_sol_FL = (re.compile(r'\n([0-9.e+-]+)[,]([0-9.e+-]+)', re.MULTILINE)).findall(source_sol_FL)
datas_sol_FL= []
for tuples_sol_FL in datalist_sol_FL:
  eachtuple_sol_FL = []
  for ab in tuples_sol_FL:
    eachtuple_sol_FL.append(float(ab))
  datas_sol_FL.append((eachtuple_sol_FL[0], eachtuple_sol_FL[1]))
data_sol_FL = np.array(datas_sol_FL)
wavelength_sol_FL = data_sol_FL[:, 0]
fluorescence = data_sol_FL[:, 1]
figure = make_subplots()
figure.add_trace(go.Scatter(x = wavelength_sol_FL, y = signal.savgol_filter(fluorescence, 53, 3)))
figure.update_xaxes(title = 'Emission Wavelength (nm)', ticks = 'outside', showgrid = False, showline = True, linecolor = 'black', range = [200, 1000], nticks = 50)
figure.update_yaxes(title = 'Fluorescence Intensity',  ticks = 'outside', showgrid = False, showline = True, linecolor = 'black')
figure.update_layout(title = f'Fluorescence Emission of {solution} at {concentration} concentration excited at {absorbance} nm')

greendye
pure
450


In [9]:
abs = wavelength_sol[np.argmax(absorbance_list)]
ems = wavelength_sol_FL[np.argmax(fluorescence)]
figure = make_subplots()
figure.add_trace(go.Scatter(x = wavelength_sol, y = signal.savgol_filter(absorbance_list, 53, 3), mode= 'lines', line_color = 'blue', name = 'Absorbance'))
figure.add_trace(go.Scatter(x = wavelength_sol_FL, y = signal.savgol_filter(fluorescence, 53, 3), mode= 'lines', line_color = 'green', name = 'Fluorescence'))
figure.update_xaxes(title = 'Wavelength (nm)', ticks = 'outside', showgrid = False, showline = True, linecolor = 'black', range = [200, 1000], nticks = 50)
figure.update_yaxes(title = 'Intensity',  ticks = 'outside', showgrid = False, showline = True, linecolor = 'black')
figure.update_layout(title = f'Excitation: {abs} nm, Emission {ems} nm')